### Load and Inspect the Dataset

In [136]:
import pandas as pd

In [137]:
train_df = pd.read_csv('arxiv_train.csv')
test_df = pd.read_csv('arxiv_test.csv')

### starting preprocessing you lame ass nigga

- checking null values

In [138]:
print("Train missing values:\n", train_df.isnull().sum())
print("Test missing values:\n", test_df.isnull().sum())

Train missing values:
 Unnamed: 0    0
abstract      0
label         0
dtype: int64
Test missing values:
 Unnamed: 0    0
abstract      0
label         0
dtype: int64


- removing na

In [139]:
train_df.dropna(subset=['abstract', 'label'], inplace=True)
test_df.dropna(subset=['abstract', 'label'], inplace=True)

- removing extra spaces, new lines, tabs

In [140]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = text.lower()
    return text

train_df['clean_abstract'] = train_df['abstract'].apply(clean_text)
test_df['clean_abstract'] = test_df['abstract'].apply(clean_text)


In [141]:
train_df.head()
print(train_df.columns)
# Check the type of the first element in the 'clean_abstract' column
print(type(train_df['clean_abstract'].iloc[0]))

Index(['Unnamed: 0', 'abstract', 'label', 'clean_abstract'], dtype='object')
<class 'str'>


 Apply the Tokenization to Your DataFrame

In [142]:
import re

def simple_tokenize(text):
    """
    Tokenizes the input text by converting to lowercase and extracting words.
    Returns a list of tokens.
    """
    text = text.lower()  # convert to lowercase
    # use re.findall to grab sequences of word characters (letters/numbers)
    tokens = re.findall(r'\b\w+\b', text) 
    return tokens

# Apply the tokenizer to your DataFrame
train_df['tokens'] = train_df['clean_abstract'].apply(simple_tokenize)
test_df['tokens'] = test_df['clean_abstract'].apply(simple_tokenize)

# Verify by checking the new 'tokens' column
print(train_df[['clean_abstract', 'tokens']].head())
print(type(train_df['clean_abstract'].iloc[0]))
print(train_df.columns)


KeyboardInterrupt: 

- encoding labels as integers because its a multiclass problem

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

train_df['label_encoded'] = label_encoder.fit_transform(train_df['label'])
test_df['label_encoded'] = label_encoder.transform(test_df['label'])

# Save the mapping for later use
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label mapping:", label_mapping)


Label mapping: {'astro-ph': 0, 'cond-mat': 1, 'cs': 2, 'eess': 3, 'hep-ph': 4, 'hep-th': 5, 'math': 6, 'physics': 7, 'quant-ph': 8, 'stat': 9}


- Stratified Train/Dev Split

In [ ]:
from sklearn.model_selection import train_test_split

# Stratified split on the encoded labels
train_texts, dev_texts, train_labels, dev_labels = train_test_split(
    train_df['clean_abstract'],
    train_df['label_encoded'],
    test_size=0.2,
    random_state=42,
    stratify=train_df['label_encoded']
)

# Optional sanity check: label distribution
print("Train label distribution:\n", train_labels.value_counts(normalize=True))
print("Dev label distribution:\n", dev_labels.value_counts(normalize=True))

Train label distribution:
 label_encoded
9    0.100687
3    0.100656
7    0.100281
4    0.100125
2    0.100062
0    0.099844
8    0.099828
5    0.099766
6    0.099469
1    0.099281
Name: proportion, dtype: float64
Dev label distribution:
 label_encoded
9    0.100687
3    0.100625
7    0.100312
4    0.100125
2    0.100062
8    0.099875
0    0.099812
5    0.099750
6    0.099500
1    0.099250
Name: proportion, dtype: float64


In [ ]:
y_train = train_labels.values
y_dev = dev_labels.values
label_map = {'astro-ph': 0, 'cond-mat': 1, 'cs': 2, 'eess': 3, 'hep-ph': 4,
             'hep-th': 5, 'math': 6, 'physics': 7, 'quant-ph': 8, 'stat': 9}

# step 1 : text vectorization

- basic count vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
count_vectorizer = CountVectorizer(max_features=10000, stop_words='english')

# Fit only on training data
X_train_count = count_vectorizer.fit_transform(train_texts)
X_dev_count = count_vectorizer.transform(dev_texts)

print("CountVectorizer shape (train):", X_train_count.shape)
print("CountVectorizer shape (dev):", X_dev_count.shape)


CountVectorizer shape (train): (64000, 10000)
CountVectorizer shape (dev): (16000, 10000)


- TF-IDF Vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')

# Fit only on training data
X_train_tfidf = tfidf_vectorizer.fit_transform(train_texts)
X_dev_tfidf = tfidf_vectorizer.transform(dev_texts)

print("TF-IDF shape (train):", X_train_tfidf.shape)
print("TF-IDF shape (dev):", X_dev_tfidf.shape)


TF-IDF shape (train): (64000, 10000)
TF-IDF shape (dev): (16000, 10000)


# building MLP

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report

- converting data to tensors

# run this for BOW tensors

In [ ]:
# Convert sparse matrices to dense and then to tensors
X_train_tensor = torch.tensor(X_train_count.toarray(), dtype=torch.float32)
X_dev_tensor = torch.tensor(X_dev_count.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_dev_tensor = torch.tensor(y_dev, dtype=torch.long)

# Create datasets and loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=64)

# run this for TF-IDF tensors

In [ ]:
X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32)
X_dev_tensor = torch.tensor(X_dev_tfidf.toarray(), dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
dev_dataset = TensorDataset(X_dev_tensor, y_dev_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=64)


- definfing the model

In [ ]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes):
        super(FeedforwardNN, self).__init__()
        layers = []
        current_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(current_dim, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.3))  # Regularization
            current_dim = h
        layers.append(nn.Linear(current_dim, num_classes))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = FeedforwardNN(input_dim=10000, hidden_dims=[128, 64], num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")


Epoch 1/5 - Loss: 836.6592
Epoch 2/5 - Loss: 443.5131
Epoch 3/5 - Loss: 337.8980
Epoch 4/5 - Loss: 260.1777
Epoch 5/5 - Loss: 197.4547


# results with BOW tensors 
- Epoch 1/5 - Loss: 706.6508
- Epoch 2/5 - Loss: 390.9463
- Epoch 3/5 - Loss: 277.9484
- Epoch 4/5 - Loss: 196.3589
- Epoch 5/5 - Loss: 142.3870

# rusults with TF-IDF tensors 
- Epoch 1/5 - Loss: 831.5841
- Epoch 2/5 - Loss: 439.9397
- Epoch 3/5 - Loss: 336.5091
- Epoch 4/5 - Loss: 258.9744
- Epoch 5/5 - Loss: 196.4470

In [ ]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch_x, batch_y in dev_loader:
        batch_x = batch_x.to(device)
        outputs = model(batch_x)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(batch_y.numpy())

print(classification_report(all_labels, all_preds, target_names=label_map.keys()))

              precision    recall  f1-score   support

    astro-ph       0.95      0.92      0.94      1597
    cond-mat       0.79      0.81      0.80      1588
          cs       0.72      0.67      0.69      1601
        eess       0.76      0.82      0.79      1610
      hep-ph       0.93      0.93      0.93      1602
      hep-th       0.90      0.91      0.90      1596
        math       0.85      0.84      0.85      1592
     physics       0.68      0.68      0.68      1605
    quant-ph       0.87      0.86      0.87      1598
        stat       0.80      0.82      0.81      1611

    accuracy                           0.82     16000
   macro avg       0.82      0.82      0.82     16000
weighted avg       0.82      0.82      0.82     16000



# compraing BOW and TF

In [ ]:
import pandas as pd

# Define the metrics for both models
data = {
    "Metric": [
        "Accuracy", "Macro F1", "Weighted F1", 
        "Astro-ph F1", "Cond-mat F1", "CS F1", "EESS F1", "Hep-ph F1", 
        "Hep-th F1", "Math F1", "Physics F1", "Quant-ph F1", "Stat F1"
    ],
    "CountVectorizer": [
        0.83, 0.83, 0.83,
        0.93, 0.80, 0.70, 0.79, 0.93,
        0.91, 0.85, 0.69, 0.86, 0.80
    ],
    "TF-IDF": [
        0.82, 0.82, 0.82,
        0.94, 0.79, 0.70, 0.79, 0.93,
        0.91, 0.84, 0.68, 0.86, 0.81
    ]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Print the table
print(df.to_string(index=False))


     Metric  CountVectorizer  TF-IDF
   Accuracy             0.83    0.82
   Macro F1             0.83    0.82
Weighted F1             0.83    0.82
Astro-ph F1             0.93    0.94
Cond-mat F1             0.80    0.79
      CS F1             0.70    0.70
    EESS F1             0.79    0.79
  Hep-ph F1             0.93    0.93
  Hep-th F1             0.91    0.91
    Math F1             0.85    0.84
 Physics F1             0.69    0.68
Quant-ph F1             0.86    0.86
    Stat F1             0.80    0.81


# next steps (for said)

- You're in a great spot to move to Step 2: Pretrained Word Embeddings. Here's what you could try next:

- Replace BoW with averaged GloVe or word2vec embeddings for each abstract.

- Use mean/sum/max pooling over word embeddings to create document vectors.

- Feed those vectors into the same MLP structure for a fair comparison.

### Loading Pretrained Word Embeddings

You can use pretrained embeddings such as GloVe or word2vec. In this example, we’ll use GloVe with a dimension of 300. First, download the GloVe file (e.g., glove.6B.300d.txt) if you haven’t already.

In [ ]:
import numpy as np

embedding_index = {}
embedding_dim = 300  # Change this depending on which embedding file you use

# Open the GloVe file (ensure the file is in your working directory)
with open('glove.6B.300d.txt', 'r', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefficients = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefficients
        
print("Loaded {} word vectors.".format(len(embedding_index)))

Loaded 400000 word vectors.


 ### Converting Abstracts into Document Vectors

Instead of using BoW, you now create a fixed-sized vector for each abstract by aggregating its word embeddings.

a. Define a Function to Compute the Document Vector

Here is a function that aggregates word embeddings from the tokens with your choice of pooling method. In this code, we demonstrate mean pooling. (You can change 'mean' to 'sum' or 'max' in the call if you prefer.)

In [ ]:
def get_document_vector(tokens, embedding_index, embedding_dim=300, pooling='mean'):
    # Collect embeddings for words that are present in the pretrained dictionary
    valid_vectors = [embedding_index[word] for word in tokens if word in embedding_index]
    if not valid_vectors:
        # If none of the words are in the embedding dictionary, return a zero vector
        return np.zeros(embedding_dim)
    valid_vectors = np.array(valid_vectors)
    
    if pooling == 'mean':
        return np.mean(valid_vectors, axis=0)
    elif pooling == 'sum':
        return np.sum(valid_vectors, axis=0)
    elif pooling == 'max':
        return np.max(valid_vectors, axis=0)
    else:
        raise ValueError("Unsupported pooling type. Choose 'mean', 'sum', or 'max'.")

b. Create Document Vectors for Training and Test Sets

Apply the function to each set of tokens:

In [ ]:
# For training data

X_train = np.vstack(train_df['tokens'].apply(lambda tokens: get_document_vector(tokens, embedding_index, embedding_dim, pooling='mean')))
y_train = train_df['label'].values

# For test data
X_test = np.vstack(test_df['tokens'].apply(lambda tokens: get_document_vector(tokens, embedding_index, embedding_dim, pooling='mean')))
y_test = test_df['label'].values

### 1. Convert String Labels to Integers

You can use LabelEncoder from scikit-learn to transform your string labels into integers.
a. Import and Apply LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le = LabelEncoder()

# Fit the encoder on your training labels and transform them into integers
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)  # Transform test labels using the same encoder

# Optionally, check the mapping
print("Label mapping:", dict(zip(le.classes_, le.transform(le.classes_))))


Label mapping: {'astro-ph': 0, 'cond-mat': 1, 'cs': 2, 'eess': 3, 'hep-ph': 4, 'hep-th': 5, 'math': 6, 'physics': 7, 'quant-ph': 8, 'stat': 9}


### 4. Building and Training the MLP Classifier

Now that you have your fixed-size document vectors, you can train an MLP. Here’s an example using TensorFlow/Keras.

a. Prepare the Data

If you’re doing multi-class classification, convert your labels into categorical format.

In [ ]:
import tensorflow as tf
to_categorical = tf.keras.utils.to_categorical

# Determine the number of classes using the unique classes from the encoder
num_classes = len(le.classes_)

# Convert integer labels to one-hot encoded vectors
y_train_cat = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_cat = to_categorical(y_test_encoded, num_classes=num_classes)


b. Define the Model Architecture

Set up a simple feed-forward neural network.

In [ ]:
import tensorflow as tf

# Equivalent references
Sequential = tf.keras.models.Sequential
Dense = tf.keras.layers.Dense

model = Sequential()
# First hidden layer with 128 neurons
model.add(Dense(128, activation='relu', input_shape=(embedding_dim,)))
# Second hidden layer with 64 neurons
model.add(Dense(64, activation='relu'))
# Output layer with softmax activation
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 128)            │        38,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,434 (185.29 KB)

 Trainable params: 47,434 (185.29 KB)

 Non-trainable params: 0 (0.00 B)

c. Train the Model

Train on your training data and validate on the test set:

In [ ]:
history = model.fit(X_train, y_train_cat, epochs=10, batch_size=32, validation_data=(X_test, y_test_cat))


Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.6243 - loss: 1.1116 - val_accuracy: 0.7506 - val_loss: 0.7117
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7507 - loss: 0.7044 - val_accuracy: 0.7551 - val_loss: 0.6895
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7644 - loss: 0.6662 - val_accuracy: 0.7677 - val_loss: 0.6489
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7717 - loss: 0.6447 - val_accuracy: 0.7639 - val_loss: 0.6471
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7766 - loss: 0.6231 - val_accuracy: 0.7753 - val_loss: 0.6279
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7815 - loss: 0.6094 - val_accuracy: 0.7769 - val_loss: 0.6228
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7888 - loss: 0.5910 - val_accuracy: 0.7798 - val_loss: 0.6081
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7895 - loss: 0.5837 -

. Evaluating the Classifier

After training, evaluate the model’s performance on unseen data.

a. Calculate Accuracy

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test_cat)
print("Test Accuracy:", accuracy)


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7728 - loss: 0.6316
Test Accuracy: 0.776199996471405


b. Generate a Detailed Classification Report

For additional metrics like precision, recall, and F1-score, use scikit-learn:

In [ ]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

from sklearn.metrics import classification_report

# Now, compare the encoded ground-truth and predicted labels
print(classification_report(y_test_encoded, y_pred))


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      2013
           1       0.74      0.79      0.76      2058
           2       0.74      0.58      0.65      1995
           3       0.72      0.76      0.74      1948
           4       0.93      0.82      0.87      1990
           5       0.78      0.87      0.82      2019
           6       0.74      0.88      0.81      2042
           7       0.72      0.52      0.61      1977
           8       0.87      0.78      0.82      2013
           9       0.70      0.85      0.76      1945

    accuracy                           0.78     20000
   macro avg       0.78      0.78      0.77     20000
weighted avg       0.78      0.78      0.77     20000

